In [19]:
import pandas as pd
import matplotlib.pylab as plt
import statsmodels.api as sm
import numpy as np
import seaborn as sns
from scipy import stats

### Raw 데이터 읽기
-  기간 : 2017년 1월 ~ 2019년 6월

In [51]:
# 가입 유저 정보
read_path = '../csv/user.csv'
user = pd.read_csv(read_path)

user['join_date'] = pd.to_datetime(user['join_date'])
user['uid'] = user['uid'].astype(str)

user.tail()

,uid,join_date,age,sex,bizlicense,device
3951551,9306356,2019-06-30 23:57:25,0,1,0,a
3951552,9306360,2019-06-30 23:57:48,0,1,0,a
3951553,9306364,2019-06-30 23:58:46,0,1,0,a
3951554,9306368,2019-06-30 23:59:18,21,2,0,w
3951555,9306372,2019-06-30 23:59:55,0,1,0,w


In [16]:
# 2017년 이후 등록 제품 정보
read_path = '../csv/product.csv'
product = pd.read_csv(read_path)

product['create_date'] = pd.to_datetime(product['create_date'])

# change type to str
for col in ['pid', 'uid']:
    
    df = product
    
    df[col] = df[col].astype(str)

product.tail()

,pid,uid,create_date,category_id,price
40264329,104478795,9306331,2019-06-30 23:59:50.969604,900100004.0,60000
40264330,104478799,5648685,2019-06-30 23:59:53.337997,320080100.0,20000
40264331,104478801,909114,2019-06-30 23:59:54.131017,999.0,75000
40264332,104478802,6291506,2019-06-30 23:59:54.694869,700350950.0,30000
40264333,104478804,4194003,2019-06-30 23:59:55.583065,600700001.0,13000


In [17]:
# 카테고리
read_path = '../csv/categories.csv'
df_category = pd.read_csv(read_path)

df_category = df_category[['category', 'name']]
df_category['category'] = df_category['category'].astype(str)

df_category.tail()

,category,name
1219,240060020,프로그래머
1220,240070040,설문조사
1221,240080060,유아유치원
1222,240100010,매장관리
1223,240100090,기타 업종


In [23]:
product_user = pd.merge(product, user, on='uid')

product_user['register_hour'] = (product_user['create_date'] - product_user['join_date']) / np.timedelta64(1,'h')

product_user.tail()

,pid,uid,create_date,category_id,price,join_date,age,sex,bizlicense,device,register_hour
12816511,104478598,9297260,2019-06-30 23:55:51.373380,320020100.0,0,2019-06-29 14:21:37,18,1,0,w,33.570659
12816512,104478692,9302607,2019-06-30 23:57:48.355590,700400.0,720000,2019-06-30 13:07:11,28,2,0,a,10.843710
12816513,104478706,9302209,2019-06-30 23:58:04.925826,600700003.0,15000,2019-06-30 11:46:40,22,2,0,i,12.190257
12816514,104478712,9298469,2019-06-30 23:58:17.244608,600700001.0,190000,2019-06-29 18:24:01,28,0,0,i,29.571179
12816515,104478795,9306331,2019-06-30 23:59:50.969604,900100004.0,60000,2019-06-30 23:52:39,18,2,0,w,0.119992


In [ ]:
user_register_hour 

In [53]:
register_1_hour = product_user[product_user['register_hour'] <= 1]

register_1_hour['category_1'] = register_1_hour['category_id'].astype(str).str[:3]

register_1_hour.tail()

/Users/quicket/python-bunjang/venv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,pid,uid,create_date,category_id,price,join_date,age,sex,bizlicense,device,register_hour,category_1
12816489,104476608,9306021,2019-06-30 23:16:48.111419,600400003.0,45000,2019-06-30 23:01:07,59,1,0,w,0.261420,600
12816492,104476535,9306108,2019-06-30 23:15:28.253927,600600005.0,130000,2019-06-30 23:12:53,16,2,0,a,0.043126,600
12816493,104476561,9306083,2019-06-30 23:15:57.057510,900320600.0,1111,2019-06-30 23:09:59,43,2,0,a,0.099460,900
12816510,104478487,9306270,2019-06-30 23:53:32.315199,910100007.0,1000,2019-06-30 23:37:59,24,1,0,w,0.259254,910
12816515,104478795,9306331,2019-06-30 23:59:50.969604,900100004.0,60000,2019-06-30 23:52:39,18,2,0,w,0.119992,900


In [57]:
register_1_hour_unique = register_1_hour.drop_duplicates(['uid'])

user_1_hour = pd.merge(user, register_1_hour_unique[['uid', 'pid']], 
                             on='uid', how='left')

user_1_hour['join_month'] = user_1_hour['join_date'].dt.strftime('%Y-%m')

user_1_hour['1_hour'] = user_1_hour['pid'].apply(lambda x : 0 if pd.isnull(x) else 1 )

# user_1_hour = pd.merge(user_1_hour, df_category, left_on='category_1', right_on='category', how='left')

# user_1_hour['category'] = user_1_hour['category'].fillna('none')
# user_1_hour['name'] = user_1_hour['name'].fillna('none')

user_1_hour.tail()

,uid,join_date,age,sex,bizlicense,device,pid,join_month,1_hour
3951551,9306356,2019-06-30 23:57:25,0,1,0,a,NaN,2019-06,0
3951552,9306360,2019-06-30 23:57:48,0,1,0,a,NaN,2019-06,0
3951553,9306364,2019-06-30 23:58:46,0,1,0,a,NaN,2019-06,0
3951554,9306368,2019-06-30 23:59:18,21,2,0,w,NaN,2019-06,0
3951555,9306372,2019-06-30 23:59:55,0,1,0,w,NaN,2019-06,0


In [59]:
user_1_hour_agg = user_1_hour.groupby(['join_month', 'age', 'sex', 'bizlicense',
                                       '1_hour'
                                      ], as_index=False).agg({'uid': 'count'})

user_1_hour_agg = user_1_hour_agg.rename(columns={'uid': 'uid_count'})

# save
save_path = '../csv/result/user_1_hour_agg.csv'
user_1_hour_agg.to_csv(save_path, index=False, mode='w', header=True)

user_1_hour_agg.tail()

,join_month,age,sex,bizlicense,1_hour,uid_count
20162,2019-06,89,2,0,0,1
20163,2019-06,91,2,1,1,1
20164,2019-06,99,2,0,1,1
20165,2019-06,114,2,0,1,1
20166,2019-06,115,2,0,0,1
